# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [42]:
from collections import Counter

from nltk import ngrams
import pandas as pd

In [43]:

bigrams = pd.read_csv("w2_.txt", sep="\t", header=None, encoding='latin-1')
bigrams.columns = ["frequency", "0", "1"]
bigrams = bigrams.dropna()
bigrams.head()

,frequency,0,1
0,275,a,a
1,31,a,aaa
2,29,a,all
3,45,a,an
4,192,a,and


In [44]:

fivegrams = pd.read_csv("fivegrams.txt", sep="\t", header=None)
fivegrams.columns = ["frequency", "0", "1", "2", "3", "4"]
fivegrams.head()

,frequency,0,1,2,3,4
0,16,a,babe,in,the,woods
1,6,a,baby,at,her,breast
2,9,a,baby,brother,or,sister
3,6,a,baby,crying,in,the
4,6,a,baby,girl,was,born


In [45]:
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)



class SpellCorrector:
    def __init__(self, grams: pd.DataFrame, n_grams: int, word_threshold: int = 50) -> None:
        self.n_grams = n_grams
        self.word_freq = Counter(grams["0"])
        for i in range(1, n_grams):
            self.word_freq.update(grams[str(i)])
            
        _to_drop = []
        for word, freq in self.word_freq.items():
            if freq < word_threshold:
                _to_drop.append(word)
        
        _stay_index = ~grams['0'].isin(_to_drop)
        for i in range(1, self.n_grams):
            _stay_index = _stay_index & (grams[str(i)].isin(_to_drop))
            
        self.grams = grams[_stay_index]
        
        self.word_freq = Counter({k: c for k, c in self.word_freq.items() if c >= word_threshold})
        self.ALL_WORDS = set(self.word_freq)
        
    
    def _get_candidates(self, word: str) -> tuple[list[str], list[str]]:
        dist1candidates = edits1(word)
        dist2candidates = set(e2 for e1 in dist1candidates for e2 in edits1(e1))
        dist2candidates = dist2candidates - dist1candidates

        dist1candidates = list(filter(lambda x: x in self.ALL_WORDS, dist1candidates))
        dist2candidates = list(filter(lambda x: x in self.ALL_WORDS, dist2candidates))
        return dist1candidates, dist2candidates
        
    def _one_word_case(self, word: str) -> str:
        first_candidates, second_candidates = self._get_candidates(word)
        word2prob = []
        for candidate in first_candidates:
            candidate_prob = self.word_freq.get(candidate) / self.word_freq.total()
            word2prob.append((candidate, candidate_prob))
        
        for candidate in second_candidates:
            candidate_prob = self.word_freq.get(candidate) / self.word_freq.total()
            candidate_prob *= 1/2
            # assumption that if 2 mistakes is 2 time worse
            word2prob.append((candidate, candidate_prob))
        
        if len(word2prob) == 0:
            return word
    
        corrected_word, prob = max(word2prob, key=lambda x: x[1])
        return corrected_word

    def correct(self, text: str) -> str:
        text = text.lower()
        words = text.split()
        
        corrected = []
        
        for i in range(min(len(words), self.n_grams-1)):
            word = words[i]
            if word in self.ALL_WORDS:
                # for now if word exists we will not change it
                corrected.append(word)
                continue
            corrected_word = self._one_word_case(word)
            corrected.append(corrected_word)
        
        for i in range(self.n_grams-1, len(words)):
            word = words[i]
            if word in self.ALL_WORDS:
                corrected.append(word)
                continue
                
            context = self.grams[self.grams["0"] == corrected[-self.n_grams + 1]]
            for j in range(1, self.n_grams-1):
                context = context[context[str(j)] == corrected[j - self.n_grams + 1]]
            
            if context.shape[0] == 0:
                corrected.append(self._one_word_case(word))
                continue
            
            first_candidates, second_candidates = self._get_candidates(word)
            word2prob = []
            for candidate in first_candidates:
                tmp = context[context[str(self.n_grams-1)] == candidate]
                if tmp.shape[0] == 0:
                    word2prob.append((candidate, 0.0))
                else:
                    tmp = tmp['frequency'].iloc[0]
                    prob = tmp / context["frequency"].sum()
                    word2prob.append((candidate, prob))
            
            for candidate in second_candidates:
                tmp = context[context[str(self.n_grams-1)] == candidate]
                if tmp.shape[0] == 0:
                    word2prob.append((candidate, 0.0))
                else:
                    tmp = tmp['frequency'].iloc[0]
                    prob = tmp / context["frequency"].sum()
                    prob /= 2
                    word2prob.append((candidate, prob))
            if len(word2prob) == 0:
                corrected.append(word)
                continue
            corrected_word, prob = max(word2prob, key=lambda x: x[1])
            if prob > 0.0:
                corrected.append(corrected_word)
            else:
                corrected.append(word)
        
        return " ".join(corrected)

In [46]:
corrector = SpellCorrector(bigrams, 2)
corrector.correct("i mde a hge corrector")

'i mde a hge correct'

In [62]:
from jellyfish import damerau_levenshtein_distance

class BigramPredictonCorrector:
    def __init__(self, bigrams: pd.DataFrame, word_threshold=50) -> None:
        self.word_freq = Counter(bigrams["0"])
        self.word_freq.update(bigrams["1"])
        self.word_threshold = word_threshold
        
        _to_drop = []
        for word, freq in self.word_freq.items():
            if freq < word_threshold:
                _to_drop.append(word)
        
        self.word_freq = Counter({k: c for k, c in self.word_freq.items() if c >= word_threshold})
        
        bigrams = bigrams[(~bigrams['1'].isin(_to_drop)) & (~bigrams['0'].isin(_to_drop))]
            
        self.ALL_WORDS = set(self.word_freq)
        self.bigrams = bigrams
    
    def correct(self, text: str):
        text = text.lower()
        tokens = text.split()
        
        # initally 'a' token will be there
        corrected = ["a"]
        
        for word in tokens:
            if word in self.ALL_WORDS:
                # print("IN DICTIONARY:", word)
                corrected.append(word)
                continue
            
            candidates: pd.DataFrame = self.bigrams[self.bigrams['0'] == corrected[-1]]
            candidates = candidates[['frequency', '1']]
            context_support = int(candidates['frequency'].sum())
            
            best_candidate = word
            best_prob = -1
            best_dist = len(word) * 2
            for _, (freq, candidate) in candidates.iterrows():
                dist = damerau_levenshtein_distance(candidate, word)
                prob = freq / context_support
                prob *= 1/(dist+1)
                
                if (dist < best_dist) or (dist == best_dist and prob > best_prob):
                    best_prob = prob
                    best_dist = dist
                    best_candidate = candidate 
                    
            corrected.append(best_candidate)
            
        return " ".join(corrected[1:])
                

corrector = BigramPredictonCorrector(bigrams)
corrector.correct("i mde a hge corrector")

'i made a huge collection'

In [48]:
# Norvig solution for further tests and comparison
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

# WORDS = Counter(words(open('big.txt').read()))
WORDS = Counter(corrector.ALL_WORDS)

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

# Justifications

I decided to implement two approaches:
- Nordvig based approach that will compute word probability using N-gramm model (`SpellCorrector`)
- N-gramm candidate approach. That will use previous words and their possible next word as candidates. And also, it will use N-gramm model for predictin word probability (`BigramPredictonCorrector`)

## SpellCorector

### Selection mechanism

Simple argmax

### Candidate model

I used same candidate model as Nordvig solution. Build edit distance of 1 and 2 of word. And check if they are in dictionary of existing words

### Language model

As language model I use bigram model, because 3-grams or more are to specific for simple typo correction and most importantly there are more data in bigrams.

I count frequency of previous word and candidate and divide by all bigrams with same previois word.

### Error model

As error model I use following:
- if edit distance is 1, probability is 1
- if edit distance is 2, probability is 0.5

It is simple and dummy approach. Probabilities here are not normalised due to argmax nature. Intuition behind: every mistake should divide our confidence in half.

## BigramPredictonCorrector

### Selection mechanism

Simple argmax

### Candidate model

Previous word in text is analyzed. And all possible continuation in bigrams dictionary are our candidates. For example, for pair `good fsh` candidate `x` will have following entry in bigrams dictionary `good x`.

### Language model

Here n-gram model is also used. Probability computed as frequency of pair `(prefix, candidate)` divided by sum of all frequencies `(prefix, x)`, where x are possible candidates

### Error model

For error model i used Damerau Levenshtein distance which adds swapping of characters with cost 1. I consider that less edit distance will be infitely more likely. So in the end least edit distance is chosen.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

Test set generation is done by following operations:
- adding random character
- swapping 2 neighbour characters
- duplicate 1 character

In [49]:
import re
import random

def introduce_errors(words: list[str], error_rate:float=0.1) -> str:
    num_errors = int(len(words) * error_rate)
    
    for _ in range(num_errors):
        index = random.randint(0, len(words) - 1)
        word = words[index]
        if len(word) > 1:
            choice = random.randint(0, 2)
            if choice == 0:
                # swap
                error_index = random.randint(0, len(word) - 2)
                word = list(word)
                word[error_index], word[error_index + 1] = word[error_index + 1], word[error_index]
                words[index] = ''.join(word)
            elif choice == 1:
                # duplicate
                error_index = random.randint(0, len(word) - 1)
                word = word[:error_index] + word[error_index] + word[error_index:]
                words[index] = word
            else:
                # random addition
                error_index = random.randint(0, len(word) - 1)
                typo = chr(ord('a') + random.randint(0, 25))
                words[index] = word[:error_index] + typo + word[error_index + 1:]
    
    
    return ' '.join(words)
introduce_errors("mein leben ist gut und schon der hund ist klein aber schon".split(), error_rate=0.5)

'mein leben gst gut und schon drj hnud ist klein aaber schou'

In [50]:
def get_test_set(noise=0.1):
    with open("big.txt") as f:
        big = f.readlines()

    big = filter(lambda x: len(x) > 2, map(lambda x: x.replace('\n', ''), big))
    x = []
    y = []
    for line in big:
        line = line.lower()
        words = re.findall(r'\w+', line)
        x.append(" ".join(words))
        text = introduce_errors(words, error_rate=noise)
        y.append(text)
    return x, y

x, y = get_test_set()

In [51]:
for i in range(3):
    print(x[i])
    print(y[i])
    print("-"*20)

the project gutenberg ebook of the adventures of sherlock holmes
the project gutenberg ebook of the adventures of sherlock homles
--------------------
by sir arthur conan doyle
by sir arthur conan doyle
--------------------
15 in our series by sir arthur conan doyle
15 in our series by sir arthur conan doyle
--------------------


In [55]:
from tqdm import trange, tqdm

def test(x, y, sample=1):
    corrector1 = SpellCorrector(bigrams, 2)
    corrector2 = BigramPredictonCorrector(bigrams)
    
    n_bad1, n_bad2, n_bad3 = 0, 0, 0
    total = 0
    
    sample_indexes = [i for i in range(len(x))]
    random.shuffle(sample_indexes)
    sample_indexes = sample_indexes[:round(sample * len(x))]
    for i in tqdm(sample_indexes):
        text = x[i]
        target = y[i].split()
        total += len(target)
        out1 = corrector1.correct(text).split()
        out2 = corrector2.correct(text).split()
        out3 = list(map(correction, text.split()))

        for j in range(len(target)):
            if target[j] != out1[j]:
                n_bad1 += 1
            if target[j] != out2[j]:
                n_bad2 += 1
            if target[j] != out3[j]:
                n_bad3 += 1
    return 1 - n_bad1/total, 1 - n_bad2/total, 1 - n_bad3/total

acc1, acc2, norvig = test(x, y, sample=0.01)

100%|██████████| 1035/1035 [06:13<00:00,  2.77it/s]


In [56]:
print(f"Norvig solution accuracy: {norvig}")
print(f"Norvig solution with bigram model accuracy: {acc1}")
print(f"Bigram prediction model accuracy {acc2}")

Norvig solution accuracy: 0.8300966569122994
Norvig solution with bigram model accuracy: 0.8989979604504744
Bigram prediction model accuracy 0.771836481333688


Here we can see that Norvig solution with Bigram model produced best result. And bigram prediction model has lowest probability. Maybe, reason that edit distance if key factor. However, if I trie to use some multiplication of edit distance and candidate probability, I will get most popular word that is not close to original.